In [1]:
import piecash
import os
import numpy as np
import pandas as pd
import inspect
from datetime import datetime

In [32]:
def checkNameForSubCategory(stringList, names=['Maciek', 'Justyna']):
    '''Checks if there is any name from names in List of Strings. 
    List is checked only from the 3rd value up to one before last. 
    
    If the name is found, it is returned. If not, np.nan is returned.
    '''
    returnString = np.nan
    for name in names:
        returnString = name if name in ",".join(stringList[2:-1]) else returnString
    
    return returnString

In [33]:
#creating a list of lists to be used as a body of DataFrame

pwd_ = %pwd
fileName_ = pwd_ + "\\gnucash_files\\finanse_sql.gnucash"

with piecash.open_book(fileName_) as book:

    transaction_list = []
    for tr in book.transactions:
        split = tr.splits
        for single_row in split:
            if single_row.account.type == "EXPENSE":
                memo = single_row.memo.strip()
                memo = memo if len(memo) > 0 else np.nan

                tempList = list(map(str, [tr.description, 
                                          tr.post_date, 
                                          memo, 
                                          single_row.account.fullname, 
                                          single_row.value, 
                                          tr.currency.mnemonic]))
                transaction_list.append(tempList)

In [34]:
#creating pandas DataFrame from created list of lists and adding columns

sourceData = transaction_list

financeFrame = pd.DataFrame(sourceData, columns=['Name', 'Date', 'Split Description', 'Account', 'Price', 'Currency'])
financeFrame = financeFrame.sort_values(['Date', 'Name'])
financeFrame['Split Description'] = financeFrame['Split Description'].replace('nan', np.nan)

finalFrame = financeFrame.copy()

#adding Product and Shop from Split Description and Name
finalFrame['Product'] = finalFrame['Split Description'].fillna(finalFrame['Name'])
cond = finalFrame['Split Description'].isnull()
finalFrame['Shop'] = np.where(cond, np.nan, finalFrame['Name'])

finalFrame['ALL_CATEGORIES'] = finalFrame['Account'].apply(lambda x: x.split(":"))

# Extracting info from Account
'''Account -> 'Wydatki:Wspólne:Zakupy:Chemia:Osobiste - Justyna:Artykuły Do Makijażu'
    2nd: Type
    3rd to OneBeforeLast: SubCategory
    Last: Category
    '''
finalFrame['Type'] = finalFrame['ALL_CATEGORIES'].apply(lambda x: x[1])
finalFrame['Category'] = finalFrame['ALL_CATEGORIES'].apply(lambda x: x[-1])
finalFrame['SubCategory'] = finalFrame['ALL_CATEGORIES'].apply(lambda x: ":".join(x[2:-1]).strip() if len(x[2:-1]) >0 else np.nan)
finalFrame['SubType'] = finalFrame['ALL_CATEGORIES'].apply(lambda x: checkNameForSubCategory(x))

# formatting Price and Date
finalFrame['Price'] = finalFrame['Price'].apply(lambda x: float(x))
finalFrame['Date'] = finalFrame['Date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

# dropping columns that are no longer needed
finalFrame = finalFrame.drop(['Name', 'Split Description', 'Account'], axis = 1)

In [35]:
finalFrame.head(30)

,Date,Price,Currency,Product,Shop,ALL_CATEGORIES,Type,Category,SubCategory,SubType
2033,2019-03-31,6.00,PLN,Pobranie opłaty okresowej,NaN,"[Wydatki, Maciek, Opłaty Bankowe]",Maciek,Opłaty Bankowe,NaN,NaN
80,2019-04-01,5.99,PLN,Pasta do zębów,Rossmann,"[Wydatki, Wspólne, Zakupy, Chemia, Dom, Łazien...",Wspólne,Jama Ustna,Zakupy:Chemia:Dom:Łazienka,NaN
0,2019-04-01,1.00,PLN,Szatnia,NaN,"[Wydatki, Wspólne, Inne]",Wspólne,Inne,NaN,NaN
82,2019-04-02,2.19,PLN,Makaron Penne 500g,Biedronka,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Suche,Zakupy:Artykuły spożywcze,NaN
83,2019-04-02,1.64,PLN,Jabłka Ligol,Biedronka,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Owoce i Warzywa,Zakupy:Artykuły spożywcze,NaN
84,2019-04-02,3.57,PLN,Banany,Biedronka,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Owoce i Warzywa,Zakupy:Artykuły spożywcze,NaN
85,2019-04-02,1.95,PLN,Cisowianka Niegazowana,Carrefour Express,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Soki i Napoje,Zakupy:Artykuły spożywcze,NaN
86,2019-04-02,3.89,PLN,Winterfresh Guma Do Żucia,Carrefour Express,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Słodycze i Chipsy,Zakupy:Artykuły spożywcze,NaN
87,2019-04-02,2.99,PLN,Mentosy,Carrefour Express,"[Wydatki, Wspólne, Zakupy, Artykuły spożywcze,...",Wspólne,Słodycze i Chipsy,Zakupy:Artykuły spożywcze,NaN
81,2019-04-02,7.00,PLN,Kanapka,Rollo,"[Wydatki, Wspólne, Zakupy, Jedzenie Na Wynos]",Wspólne,Jedzenie Na Wynos,Zakupy,NaN
